In [33]:
import pandas as pd
import pyodbc
from sqlpush.sqlstd import *
from sqlpush.config import *

In [49]:
# how to fuzzy match : 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
fuzz.token_sort_ratio("Catherine Gitau M.", "Gitau Catherine")
fuzz.partial_ratio("Catherine M. Gitau","Catherine Gitau")

94

In [35]:
# Prepare connection
cursor, conn = connectdb(config)

In [70]:
# let's define all the needed functions

def getdata(cursor):
    query = "SELECT * FROM belgium_addresses "
    cursor.execute(query)
    tablecol = [i[0] for i in cursor.fetchall()]
    return tablecol


def get_street_id_from_postcode(cursor, postcode):
    query = "SELECT street_id FROM belgium_streets WHERE postcode = '%s'" %(postcode)
    cursor.execute(query)
    tablecol = [i[0] for i in cursor.fetchall()]
    return tablecol

def get_language(streetname):
    streetname = streetname.lower()
    language = "None"
    if "straat" in streetname :
        language = "NL"
    elif "laan" in streetname :
        language = "NL"
    elif "rue" in streetname :
        language = "FR"
    elif "avenue" in streetname :
        language = "FR"
    elif "boulevard" in streetname :
        language = "FR"
    elif "square " in streetname :
        language = "FR"
    elif "allées" in streetname :
        language = "FR"
    else : 
        language = "None"
    return language

def get_street_names_from_postcode_lg(cursor, postcode, language):
    if language == "FR" :
        query = "SELECT streetname_fr FROM belgium_streets WHERE postcode = '%s'" %(postcode)
        cursor.execute(query)
        tablecol = [i[0] for i in cursor.fetchall()]
    elif language == "NL" :
        query = "SELECT streetname_nl FROM belgium_streets WHERE postcode = '%s'" %(postcode)
        cursor.execute(query)
        tablecol = [i[0] for i in cursor.fetchall()]
    else : 
        query = "SELECT streetname_fr, streetname_nl FROM belgium_streets WHERE postcode = '%s'" %(postcode)
        cursor.execute(query)
        tablecol = cursor.fetchall()
    return tablecol

# I need a function that will take a list of names as input and a search term
# will build another list with fuzzy match score, and return the value with the highest score
def get_best_matching(list_of_streets, term) :
    scores = []
    for element in list_of_streets : 
        scores = scores + fuzz.token_sort_ratio(element, term)
    return scores

# takes as input a 2D list, outupts a 1D list
def transform_list(street_n_list) : 
    total_list = []
    for element in street_n_list : 
        total_list = total_list + element[0] + element[1]

def get_best_address_given_scores(scores, list_of_streets):
    max_value = max(scores) 
    max_index = scores. index(max_value)
    return list_of_streets[max_index]

In [56]:
# let's implement the algorithm

# take the inputs : postcode, streetname and house number
postcode = input("Ener postcode : ")
streetname = input("Enter streetname : ")
house_number = input("Enter house number : ")

In [66]:
street_ids = get_street_id_from_postcode(cursor, "1040")

In [ ]:
if len(street_ids)>0 :
    language = get_language(streetname)
    if language == "None" : 
        sn_none = get_street_names_from_postcode_lg(cursor, postcode, language)
        sn_none = transform_list(sn_none)
        scores = get_best_matching(sn_none, streetname)
    else : 
        sn = get_street_names_from_postcode_lg(cursor, postcode, language)
        scores = get_best_matching(sn, streetname)
    